In [10]:
import pickle
import pandas as pd
import numpy as np
from psm_causal_effects import psm_causal_effects

# read data
with open('data.dat') as f:
    data, subjects = pickle.load(f)
f.close()



In [12]:
import scipy as scp

#sleep on mood
confound_names_s2m = ['dow', 'mean_temp_prev', 'clear_prev', 'act_prev','daytype','stress_prev','energy_prev','focus_prev']
nSubj = len(data)
s2m_corr = np.zeros((nSubj, len(confound_names_s2m)))
s2m_pvals = np.zeros((nSubj, len(confound_names_s2m)))
                     
for i in range(nSubj):
    treatment = np.array(data[i]['quality'])
    confound = np.array(data[i][confound_names_s2m])
    tnans  = np.isnan(treatment)
    for j in range(confound.shape[1]):
        nas = np.logical_or(tnans, np.isnan(confound[:, j]))
        s2m_corr[i, j], s2m_pvals[i, j] = scp.stats.pearsonr(treatment[~nas], confound[~nas, j])

#mood on sleep   
confound_names_m2s = ['dow', 'mean_temp_prev2', 'clear_prev2', 'act_prev2','daytype','stress_prev2','energy_prev2','focus_prev2']
m2s_corr = np.zeros((nSubj, len(confound_names_m2s)))
m2s_pvals = np.zeros((nSubj, len(confound_names_m2s)))
                     
                     
#i=0
for i in range(nSubj):
    treatment = np.array(data[i]['mood_prev'])
    confound = np.array(data[i][confound_names_m2s])
    tnans  = np.isnan(treatment)
    for j in range(confound.shape[1]):
        nas = np.logical_or(tnans, np.isnan(confound[:, j]))
        m2s_corr[i, j], m2s_pvals[i, j] = scp.stats.pearsonr(treatment[~nas], confound[~nas, j])
        
        

        


In [13]:
confound_names_s2m = ['day of the week', 'mean temperature (prev day)', 'clear sky (prev day)', 'activity (previous day)',\
                      'day type','stress (previous day)','energy (previous day)','focus (previous day)']


s2m_mean = np.nanmean(s2m_corr, axis=0)
s2m_std = np.nanstd(s2m_corr, axis=0)
s2m_nsign = np.sum(s2m_pvals<0.05, axis=0)
s2m_mat = np.vstack((s2m_mean, s2m_std, s2m_nsign)).T

s2m_df = pd.DataFrame(data=s2m_mat, columns=['mean', 'std', '#p<0.05'],  index=confound_names_s2m)

confound_names_m2s = ['day of the week', 'mean temperature (prev day)', 'clear sky (prev day)', 'activity (previous day)',\
                      'day type','stress (previous day)','energy (previous day)','focus (previous day)']

m2s_mean = np.nanmean(m2s_corr, axis=0)
m2s_std = np.nanstd(m2s_corr, axis=0)
m2s_nsign = np.sum(m2s_pvals<0.05, axis=0)
m2s_mat = np.vstack((m2s_mean, m2s_std, m2s_nsign)).T

m2s_df = pd.DataFrame(data=m2s_mat, columns=['mean', 'std', '#p<0.05'],  index=confound_names_m2s)


C:\Users\Sofia\Anaconda3\envs\ana41py2dot7\lib\site-packages\ipykernel\__main__.py:6: RuntimeWarning: invalid value encountered in less
C:\Users\Sofia\Anaconda3\envs\ana41py2dot7\lib\site-packages\ipykernel\__main__.py:15: RuntimeWarning: invalid value encountered in less


In [14]:
decimals = pd.Series([3, 3, 0], index=['mean', 'std', '#p<0.05'])
m2s_df.round(decimals)


,mean,std,#p<0.05
day of the week,0.058,0.170,20.0
mean temperature (prev day),-0.019,0.195,25.0
clear sky (prev day),0.003,0.192,17.0
activity (previous day),0.018,0.189,15.0
day type,0.066,0.199,20.0
stress (previous day),0.296,0.233,110.0
energy (previous day),0.220,0.241,77.0
focus (previous day),0.248,0.227,80.0


In [20]:
np.nanmedian(m2s_corr[:, 6])

0.21301241555773331